## Chuẩn bị dữ liệu ASR với bộ 28k Vietnamese Voice

Notebook này thực hiện các bước:

1. Tải bộ dữ liệu `natmin322/28k_vietnamese_voice_augmented_of_VigBigData` từ Hugging Face.
2. Chuẩn hóa cấu trúc dữ liệu cho bài toán ASR:
   - Chuẩn audio về sampling rate 16 kHz, mono (dùng cột `audio` của Hugging Face Datasets).
   - Chuẩn hóa trường transcript (cột `sentence`).
3. Gộp các split train (`train_1` → `train_5`) thành một tập train lớn.
4. Chia lại thành `train` và `validation` (90% / 10%).
5. Giữ nguyên split `test` gốc của dataset.
6. Lưu lại các split dưới dạng Hugging Face Dataset bằng `save_to_disk` để dùng cho huấn luyện ASR sau này.


In [ ]:
# Cài đặt thư viện cần thiết (chạy một lần trong môi trường mới)
# Nếu đã có rồi thì có thể bỏ qua cell này
%pip install -q datasets soundfile librosa tqdm torchcodec

from datasets import load_dataset, concatenate_datasets, Audio
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import json

# Thư mục lưu dataset đã xử lý
OUTPUT_DIR = Path("./data/asr_28k")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Thư mục lưu dữ liệu:", OUTPUT_DIR.resolve())
a
# 1. Tải dataset 28k từ Hugging Face
print("\n[1/4] Đang tải dataset 28k Vietnamese Voice từ Hugging Face...")
vig_dataset = load_dataset("natmin322/28k_vietnamese_voice_augmented_of_VigBigData")
print("Các splits có sẵn:", list(vig_dataset.keys()))

# 2. Gộp các split train_1 → train_5
print("\n[2/4] Đang gộp các split train_1 → train_5...")
train_splits = [f"train_{i}" for i in range(1, 6)]
train_datasets = [vig_dataset[split] for split in train_splits]

vig_train_all = concatenate_datasets(train_datasets)
vig_test = vig_dataset["test"]

print(f"Số mẫu train (gộp): {len(vig_train_all)}")
print(f"Số mẫu test: {len(vig_test)}")

# 3. Chuẩn hóa cấu trúc cho ASR: audio (16 kHz), transcript (lowercase, strip)
print("\n[3/4] Đang chuẩn hóa cấu trúc dữ liệu cho ASR...")

def prepare_asr_example(example):
    """Chuẩn hóa 1 sample cho bài toán ASR."""
    # Cột `audio` của Hugging Face đã chứa array và sampling_rate
    audio = example["audio"]
    transcript = example["sentence"]

    return {
        "audio": audio,
        "transcript": str(transcript).lower().strip(),
        "duration": float(audio["duration"]),
    }

# Áp dụng cho train và test
vig_train_processed = vig_train_all.map(
    prepare_asr_example,
    remove_columns=[col for col in vig_train_all.column_names if col not in ["audio", "sentence"]],
    num_proc=4,
)

vig_test_processed = vig_test.map(
    prepare_asr_example,
    remove_columns=[col for col in vig_test.column_names if col not in ["audio", "sentence"]],
    num_proc=4,
)

# Đảm bảo cột audio được cast về Audio với sampling_rate = 16 kHz
vig_train_processed = vig_train_processed.cast_column("audio", Audio(sampling_rate=16000))
vig_test_processed = vig_test_processed.cast_column("audio", Audio(sampling_rate=16000))

print("Hoàn tất chuẩn hóa. Ví dụ 1 sample train:")
print({k: vig_train_processed[0][k] for k in ["transcript", "duration"]})

# 4. Chia train thành train/validation (90% / 10%)
print("\n[4/4] Đang chia train thành train/validation (90% / 10%)...")

split_dict = vig_train_processed.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dict["train"]
val_dataset = split_dict["test"]

total_train = len(train_dataset)
total_val = len(val_dataset)
total_test = len(vig_test_processed)

print(f"Số mẫu train: {total_train}")
print(f"Số mẫu val:   {total_val}")
print(f"Số mẫu test:  {total_test}")

# 5. Lưu dataset đã xử lý xuống đĩa bằng save_to_disk
print("\nĐang lưu các split đã xử lý xuống đĩa...")

train_path = OUTPUT_DIR / "train"
val_path = OUTPUT_DIR / "val"
test_path = OUTPUT_DIR / "test"

train_dataset.save_to_disk(str(train_path))
val_dataset.save_to_disk(str(val_path))
vig_test_processed.save_to_disk(str(test_path))

print("Đã lưu:")
print("  - Train:", train_path.resolve())
print("  - Val:  ", val_path.resolve())
print("  - Test: ", test_path.resolve())

# 6. Lưu thống kê cơ bản vào file JSON
print("\nĐang ghi thống kê dữ liệu vào stats.json...")

train_transcripts = [ex["transcript"] for ex in train_dataset]
train_lengths = [len(t) for t in train_transcripts]
train_durations = [ex["duration"] for ex in train_dataset]

stats = {
    "train_samples": total_train,
    "val_samples": total_val,
    "test_samples": total_test,
    "avg_transcript_length": float(sum(train_lengths) / len(train_lengths)),
    "min_transcript_length": int(min(train_lengths)),
    "max_transcript_length": int(max(train_lengths)),
    "avg_duration": float(sum(train_durations) / len(train_durations)),
    "min_duration": float(min(train_durations)),
    "max_duration": float(max(train_durations)),
}

with open(OUTPUT_DIR / "stats.json", "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

print("Hoàn thành chuẩn bị dữ liệu ASR cho bộ 28k Vietnamese Voice.")
print("Thống kê chính:")
for k, v in stats.items():
    print(f"  {k}: {v}")


^C
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'datasets'

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 1.1.5 which is incompatible.


## Phiên bản đầy đủ tiền xử lý ASR (28k) theo các bước trong thuyết minh

Phần dưới đây hiện thực pipeline tiền xử lý đầy đủ cho bộ `natmin322/28k_vietnamese_voice_augmented_of_VigBigData` dùng cho ASR:

1. **Chuẩn hóa định dạng**: audio về 16 kHz, mono, chuẩn hóa biên độ (tránh clipping).
2. **Lọc nhiễu, cắt im lặng, loại bỏ mẫu lỗi**: loại bỏ các mẫu quá ngắn/dài, nhiều im lặng, clipping, RMS quá thấp.
3. **(Không cân bằng nhãn)**: bộ 28k không có nhãn cảm xúc, nên bước cân bằng nhãn không áp dụng cho ASR.
4. **Tăng cường dữ liệu (data augmentation)**: áp dụng cho tập train (time-stretch, pitch-shift, thêm noise, thay đổi volume) để tăng đa dạng tín hiệu.
5. **Chia dữ liệu**:
   - Tập train/val được tạo từ các split `train_1` → `train_5` sau khi lọc và augment.
   - Tập test giữ nguyên từ split `test` của dataset (chỉ áp dụng chuẩn hóa + lọc nhiễu, không augment).
6. **Lưu kết quả**: lưu thành Hugging Face Dataset tại `./data/asr_28k/{train,val,test}` và ghi thống kê `stats.json`.

Sau khi chạy xong cell code bên dưới, bạn có thể sử dụng trực tiếp các thư mục này trong notebook `finetune_ASR_28k.ipynb`.


In [ ]:
# PIPELINE TIỀN XỬ LÝ ĐẦY ĐỦ CHO ASR (28K DATASET)

%pip install -q datasets librosa soundfile tqdm noisereduce scikit-learn torchcodec

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import noisereduce as nr

from scipy import signal
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from datasets import load_dataset, Dataset, Audio
import json
import random
import librosa.effects as effects

# ==========================
# 1. CHUẨN HÓA + TẠO normalized_data_28k
# ==========================

print("=" * 60)
print("1. CHUẨN HÓA ĐỊNH DẠNG CHO 28K DATASET")
print("=" * 60)

vig_dataset = load_dataset("natmin322/28k_vietnamese_voice_augmented_of_VigBigData")
print("Các splits có sẵn:", list(vig_dataset.keys()))

normalized_data_28k = []

TARGET_SR = 16000

for split_name in ["train_1", "train_2", "train_3", "train_4", "train_5", "test"]:
    print(f"\nProcessing split: {split_name} ...")
    split_data = vig_dataset[split_name]

    for idx, ex in enumerate(tqdm(split_data, desc=split_name)):
        audio = ex["audio"]  # {'array', 'sampling_rate', 'path', ...}
        y = audio["array"]
        sr = audio["sampling_rate"]

        # Đảm bảo mono
        if y.ndim > 1:
            y = librosa.to_mono(y)

        # Resample về 16 kHz nếu cần
        if sr != TARGET_SR:
            y = librosa.resample(y, orig_sr=sr, target_sr=TARGET_SR)
            sr = TARGET_SR

        # Normalize amplitude (tránh clipping)
        max_val = np.abs(y).max()
        if max_val > 0.95:
            y = y / max_val * 0.95

        normalized_data_28k.append(
            {
                "audio_array": y,
                "sampling_rate": sr,
                "transcript": str(ex["sentence"]).lower().strip(),
                "duration": len(y) / sr,
                "split": split_name,
            }
        )

print(f"\nĐã chuẩn hóa {len(normalized_data_28k)} samples từ 28k dataset.")

# ==========================
# 2. LỌC NHIỄU, CẮT IM LẶNG, LOẠI MẪU LỖI
# ==========================

print("=" * 60)
print("2. LỌC NHIỄU, CẮT IM LẶNG, LOẠI MẪU LỖI CHO ASR")
print("=" * 60)


def detect_silence(audio, sr, threshold_db=-40):
    rms = librosa.feature.rms(y=audio)[0]
    rms_db = librosa.power_to_db(rms**2)
    silence_mask = rms_db < threshold_db
    silence_ratio = np.sum(silence_mask) / len(silence_mask)
    return silence_ratio


def remove_silence(audio, sr, frame_length=2048, hop_length=512, top_db=20):
    audio_trimmed, _ = librosa.effects.trim(
        audio,
        top_db=top_db,
        frame_length=frame_length,
        hop_length=hop_length,
    )
    return audio_trimmed


def reduce_noise(audio, sr, stationary=False):
    try:
        audio_clean = nr.reduce_noise(
            y=audio,
            sr=sr,
            stationary=stationary,
        )
        return audio_clean
    except Exception:
        b, a = signal.butter(3, 80, "hp", fs=sr)
        audio_clean = signal.filtfilt(b, a, audio)
        return audio_clean


def check_audio_quality(audio, sr, min_duration=0.5, max_duration=30):
    duration = len(audio) / sr

    if duration < min_duration or duration > max_duration:
        return False, f"Invalid duration: {duration:.2f}s"

    silence_ratio = detect_silence(audio, sr)
    if silence_ratio > 0.8:
        return False, f"Too much silence: {silence_ratio:.2%}"

    max_val = np.abs(audio).max()
    if max_val > 0.99:
        return False, f"Audio clipping detected: {max_val:.3f}"

    rms = librosa.feature.rms(y=audio)[0].mean()
    if rms < 0.005:
        return False, f"RMS too low: {rms:.4f}"

    return True, "OK"


def preprocess_audio_pipeline(audio_array, sr, reduce_noise_flag=True):
    audio_processed = remove_silence(audio_array, sr)

    if reduce_noise_flag:
        audio_processed = reduce_noise(audio_processed, sr)

    max_val = np.abs(audio_processed).max()
    if max_val > 0:
        audio_processed = audio_processed / max_val * 0.95

    return audio_processed


cleaned_data_28k = []
removed_samples_28k = []

for idx, sample in enumerate(tqdm(normalized_data_28k, desc="Cleaning 28k")):
    audio_array = sample["audio_array"]
    sr = sample["sampling_rate"]

    is_valid, reason = check_audio_quality(audio_array, sr)

    if not is_valid:
        removed_samples_28k.append(
            {"idx": idx, "reason": reason, "split": sample["split"]}
        )
        continue

    try:
        audio_cleaned = preprocess_audio_pipeline(audio_array, sr)
        is_valid_after, reason_after = check_audio_quality(audio_cleaned, sr)

        if is_valid_after:
            sample["audio_array"] = audio_cleaned
            sample["duration"] = len(audio_cleaned) / sr
            cleaned_data_28k.append(sample)
        else:
            removed_samples_28k.append(
                {
                    "idx": idx,
                    "reason": f"After processing: {reason_after}",
                    "split": sample["split"],
                }
            )
    except Exception as e:
        removed_samples_28k.append(
            {
                "idx": idx,
                "reason": f"Processing error: {str(e)}",
                "split": sample["split"],
            }
        )

print(f"\nĐã làm sạch {len(cleaned_data_28k)} samples")
print(f"Đã loại bỏ {len(removed_samples_28k)} samples")

# ==========================
# 3. TÁCH TRAIN/TEST VÀ AUGMENT CHỈ TRÊN TRAIN
# ==========================

print("=" * 60)
print("3. TÁCH TRAIN/TEST VÀ TĂNG CƯỜNG DỮ LIỆU CHO TRAIN")
print("=" * 60)

cleaned_train = [s for s in cleaned_data_28k if s["split"] != "test"]
cleaned_test = [s for s in cleaned_data_28k if s["split"] == "test"]

print(f"Số mẫu train (sau làm sạch): {len(cleaned_train)}")
print(f"Số mẫu test  (sau làm sạch): {len(cleaned_test)}")


class AudioAugmentationASR:
    @staticmethod
    def time_stretch(audio, sr, rate_range=(0.9, 1.1)):
        rate = random.uniform(*rate_range)
        return effects.time_stretch(audio, rate=rate)

    @staticmethod
    def pitch_shift(audio, sr, n_steps_range=(-1, 1)):
        n_steps = random.randint(*n_steps_range)
        return effects.pitch_shift(audio, sr=sr, n_steps=n_steps)

    @staticmethod
    def add_gaussian_noise(audio, snr_db_range=(15, 30)):
        snr_db = random.uniform(*snr_db_range)
        signal_power = np.mean(audio**2)
        noise_power = signal_power / (10 ** (snr_db / 10))
        noise = np.random.normal(0, np.sqrt(noise_power), len(audio))
        return audio + noise

    @staticmethod
    def volume_scale(audio, scale_range=(0.8, 1.2)):
        scale = random.uniform(*scale_range)
        return audio * scale

    @staticmethod
    def apply_all(audio, sr, prob=0.5):
        augmented = audio.copy()

        if random.random() < prob:
            augmented = AudioAugmentationASR.time_stretch(augmented, sr)

        if random.random() < prob:
            augmented = AudioAugmentationASR.pitch_shift(augmented, sr)

        if random.random() < prob:
            augmented = AudioAugmentationASR.add_gaussian_noise(augmented)

        if random.random() < prob:
            augmented = AudioAugmentationASR.volume_scale(augmented)

        max_val = np.abs(augmented).max()
        if max_val > 0:
            augmented = augmented / max_val * 0.95

        return augmented


def augment_train_dataset(train_list, augmentation_factor=1.0):
    """Tăng cường dữ liệu cho train (factor=1.0 nghĩa là tạo thêm ~100% số mẫu thiểu số)."""
    if augmentation_factor <= 0:
        return train_list

    augmented = list(train_list)

    target_extra = int(len(train_list) * augmentation_factor)
    print(f"Sẽ tạo thêm khoảng {target_extra} mẫu augment cho train.")

    for _ in tqdm(range(target_extra), desc="Augmenting train"):
        sample = random.choice(train_list)
        audio_array = sample["audio_array"]
        sr = sample["sampling_rate"]

        aug_audio = AudioAugmentationASR.apply_all(audio_array, sr)
        new_sample = sample.copy()
        new_sample["audio_array"] = aug_audio
        new_sample["duration"] = len(aug_audio) / sr
        new_sample["is_augmented"] = True
        augmented.append(new_sample)

    return augmented


augmented_train = augment_train_dataset(cleaned_train, augmentation_factor=0.5)
print(f"\nTrain trước augment: {len(cleaned_train)}")
print(f"Train sau augment:   {len(augmented_train)}")

# ==========================
# 4. CHIA TRAIN/VAL VÀ LƯU HF DATASET
# ==========================

print("=" * 60)
print("4. CHIA TRAIN/VAL VÀ LƯU DỮ LIỆU CHO ASR")
print("=" * 60)

train_df = pd.DataFrame(augmented_train)

train_split_df, val_split_df = train_test_split(
    train_df,
    test_size=0.1,
    random_state=42,
)

print(f"Số mẫu train: {len(train_split_df)}")
print(f"Số mẫu val:   {len(val_split_df)}")
print(f"Số mẫu test:  {len(cleaned_test)}")


def records_to_asr_dataset(records):
    audio_arrays = [r["audio_array"] for r in records]
    transcripts = [r["transcript"] for r in records]
    durations = [r["duration"] for r in records]

    ds = Dataset.from_dict(
        {
            "audio": audio_arrays,
            "transcript": transcripts,
            "duration": durations,
        }
    )
    ds = ds.cast_column("audio", Audio(sampling_rate=16000))
    return ds


train_ds = records_to_asr_dataset(train_split_df.to_dict("records"))
val_ds = records_to_asr_dataset(val_split_df.to_dict("records"))
test_ds = records_to_asr_dataset(cleaned_test)

OUTPUT_DIR = Path("./data/asr_28k")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

train_ds.save_to_disk(str(OUTPUT_DIR / "train"))
val_ds.save_to_disk(str(OUTPUT_DIR / "val"))
test_ds.save_to_disk(str(OUTPUT_DIR / "test"))

print("\nĐã lưu dữ liệu ASR đã tiền xử lý đầy đủ vào ./data/asr_28k")

# ==========================
# 5. THỐNG KÊ
# ==========================

train_durations = [ex["duration"] for ex in train_ds]

stats = {
    "train_samples": len(train_ds),
    "val_samples": len(val_ds),
    "test_samples": len(test_ds),
    "total_samples": len(train_ds) + len(val_ds) + len(test_ds),
    "avg_duration": float(sum(train_durations) / len(train_durations)),
    "min_duration": float(min(train_durations)),
    "max_duration": float(max(train_durations)),
}

with open(OUTPUT_DIR / "stats.json", "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

print("\nThống kê chính:")
for k, v in stats.items():
    print(f"  {k}: {v}")

